# Import Packages

In [2]:
import os
import gpxpy
import gpxpy.gpx
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString
import matplotlib.pyplot as plt
from datetime import datetime

# Convert and Calculate

In [5]:
def GPX_to_GeoDataFrame(gpx_file,CRS='EPSG:4326'):
    Point_Layer = {
    'Lat':[],
    'Lon':[],
    'Z':[],
    'Time':[],
    'HR':[]
    }
    gpx_file = open(gpx_file,'r')
    gpx = gpxpy.parse(gpx_file)
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                Point_Layer['Lat'].append(point.latitude)
                Point_Layer['Lon'].append(point.longitude)
                Point_Layer['Z'].append(point.elevation)
                Point_Layer['Time'].append(point.time)
                for ext in point.extensions:
                    for extchild in list(ext):
                        if extchild.tag[-2:] == 'hr':
                            Point_Layer['HR'].append(extchild.text)
                            
    df = pd.DataFrame(Point_Layer)

    Points = gpd.GeoDataFrame(df,
                           geometry=gpd.points_from_xy(df['Lon'],df['Lat']),
                           crs='EPSG:4326'
                           )
    Points = Points.to_crs(CRS)
    
    Points['Time'] = pd.DatetimeIndex(Points['Time'])
    Points['Seconds']=Points['Time'].diff().dt.seconds
    Points['Delta_Z']=Points['Z'].diff()
    Points2 = Points.shift()
    Points['Key']=Points.index
    Points2['Key']=Points2.index
    Points_to_Lines=Points.append(Points2)
    Points_to_Lines=Points_to_Lines.sort_values(by='Key')
    Temp = Points_to_Lines[2:].groupby('Key').geometry.apply(lambda x: LineString(x.tolist()))
    Lines = Points_to_Lines[2:].copy()
    Lines.geometry=Temp
    Lines['Distance']=Lines.length
    Lines['Speed']=Lines['Distance']/Lines['Seconds']
    Lines=Lines.drop(['Time','Lat','Lon'],axis=1)
    return(Lines)

# Read and Save Files

In [ ]:
T1 = datetime.now()
for path,dirs,files in os.walk('Inputs/'):
    for f in files:
        gpx_file = path+f
        Lines=GPX_to_GeoDataFrame(gpx_file,CRS='EPSG:3005')
        Lines.to_file('Outputs/'+f.split('.')[0]+'.shp')
print(datetime.now()-T1)    
Lines.plot(column='Speed')